In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
stance = pd.read_csv("//Users//kelvin//Downloads//clean.csv", header=0,)
stance.head(10)

,Unnamed: 0,Headline,unrelated,agree,disagree,discuss,Body,Number.Of.Common.Words,Number.Of.Words.In.Body,Number.Of.Words.In.Headline,Stance
0,1,police find mass graves with at least bodies n...,1,0,0,0,danny boyle is directing the untitled film set...,2,198,17,unrelated
1,2,hundreds of palestinians flee floods in gaza a...,0,1,0,0,hundreds of palestinians were evacuated from t...,10,429,11,agree
2,3,christian bale passes on role of steve jobs ac...,1,0,0,0,year old moscow resident was hospitalized with...,4,187,17,unrelated
3,4,hbo and apple in talks for month apple tv stre...,1,0,0,0,reuters a canadian soldier was shot at the can...,2,79,14,unrelated
4,5,spider burrowed through tourist s stomach and ...,0,0,1,0,fear not arachnophobes the story of bunbury s ...,9,627,11,disagree
5,6,nasa confirms earth will experience days of to...,0,1,0,0,thousands of people have been duped by a fake ...,16,425,16,agree
6,7,accused boston marathon bomber severely injure...,1,0,0,0,a british fighter who travelled to iraq to sto...,2,961,14,unrelated
7,8,identity of isis terrorist known as jihadi joh...,1,0,0,0,adding to apple s ios launch troubles a report...,2,315,10,unrelated
8,9,banksy arrested real identity revealed is the ...,0,1,0,0,if you ve seen a story floating around on your...,8,347,12,agree
9,10,british aid worker confirmed murdered by isis,1,0,0,0,the british islamic state militant who has fea...,4,238,7,unrelated


In [3]:
stance.shape

(49972, 11)

In [4]:
stance.drop('Unnamed: 0' , axis=1, inplace=True)

In [5]:
stance.head(1)

,Headline,unrelated,agree,disagree,discuss,Body,Number.Of.Common.Words,Number.Of.Words.In.Body,Number.Of.Words.In.Headline,Stance
0,police find mass graves with at least bodies n...,1,0,0,0,danny boyle is directing the untitled film set...,2,198,17,unrelated


In [8]:
# examine the class distribution
stance.Stance.value_counts().sort_index()


agree         3678
disagree       840
discuss       8909
unrelated    36545
Name: Stance, dtype: int64

In [8]:

# how to define X and y (from the SMS data) for use with COUNTVECTORIZER
X = stance.Body
y = stance.label_num
print(X.shape)
print(y.shape)

(49972,)
(49972,)


In [10]:
stance.head(3)

,Headline,unrelated,agree,disagree,discuss,Body,Number.Of.Common.Words,Number.Of.Words.In.Body,Number.Of.Words.In.Headline,Stance,label_num
0,police find mass graves with at least bodies n...,1,0,0,0,danny boyle is directing the untitled film set...,2,198,17,unrelated,3
1,hundreds of palestinians flee floods in gaza a...,0,1,0,0,hundreds of palestinians were evacuated from t...,10,429,11,agree,0
2,christian bale passes on role of steve jobs ac...,1,0,0,0,year old moscow resident was hospitalized with...,4,187,17,unrelated,3


In [7]:
# convert label to a numerical variable
stance['label_num'] = stance.Stance.map({'agree':0, 'disagree':1, 'discuss':2, 'unrelated':3})

In [159]:
news_bias_con = stance[(stance.label_num==0) | (stance.label_num==1)]

In [163]:
news_bias_con.shape

(24964, 3)

In [161]:
X = news_bias_con.Headline
y = news_bias_con.label_num

In [9]:
# split X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(37479,)
(12493,)
(37479,)
(12493,)


/Users/kelvin/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
# import and instantiate CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [11]:
# learn training data vocabulary, then use it to create a document-term matrix
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

In [12]:
# equivalently: combine fit and transform into a single step
X_train_dtm = vect.fit_transform(X_train)

In [13]:
# examine the document-term matrix
X_train_dtm

<37479x22273 sparse matrix of type '<class 'numpy.int64'>'
	with 7104813 stored elements in Compressed Sparse Row format>

In [ ]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<12493x22273 sparse matrix of type '<class 'numpy.int64'>'
	with 2360224 stored elements in Compressed Sparse Row format>

In [19]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [20]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 98.6 ms, sys: 14.5 ms, total: 113 ms
Wall time: 115 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [22]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.48595213319458896

In [23]:
# null acuracy
y_test.value_counts().head(1) / y_test.shape

3    0.726727
Name: label_num, dtype: float64

In [24]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[ 442,  144,  227,  110],
       [  29,  140,   30,   23],
       [ 196,  189, 1451,  433],
       [1714,  938, 2389, 4038]])

In [25]:
# print the classification report
print(metrics.classification_report(y_test, y_pred_class))

             precision    recall  f1-score   support

          0       0.19      0.48      0.27       923
          1       0.10      0.63      0.17       222
          2       0.35      0.64      0.46      2269
          3       0.88      0.44      0.59      9079

avg / total       0.72      0.49      0.53     12493



In [26]:
# print message text for the false positives (ham incorrectly classified as spam)
X_test[y_test < y_pred_class]

48435    hbo plans to launch its stand alone net video ...
28405    a man dubbed the guru in bling allegedly convi...
10084    the man who scaled the white house fence and w...
41858    a recent report from cosmic book news stated t...
17361    no suspected cases of ebola have been found in...
18252    a report that says a family in the town of pur...
11613    matt taibbi takes a leave of absence from firs...
36329    when apple introduced its apple watch in septe...
21740    a former pga tour player claims tiger woods ha...
11004    the pentagon confirmed that a cache of weapons...
30653    sources in the kurdish underground in syria de...
34351    the cheese loving tyrant was last seen in publ...
12046    gill rosenberg the canadian israeli volunteer ...
5520     a year old member of the isis terror group has...
29932    the frightening story about a spider that repo...
16799    homeland security secretary jeh johnson says i...
43956    that s how much walmart reported pulled in per.

In [ ]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

In [ ]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

In [27]:
from sklearn.ensemble import RandomForestClassifier
#forest = RandomForestClassifier(n_estimators = 100)
forest = RandomForestClassifier(max_depth = 10, min_samples_split=2, n_estimators = 100, random_state = 1)
my_forest = forest.fit(X_train_dtm, y_train)

In [28]:
print(my_forest.score(X_train_dtm, y_train))

0.75060700659


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(X_train_dtm, y_train)

In [29]:
# import and instantiate a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [30]:
# train the model using X_train_dtm
%time logreg.fit(X_train_dtm, y_train)

CPU times: user 3min 43s, sys: 1.81 s, total: 3min 45s
Wall time: 1min 55s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [31]:
# make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)

In [32]:
# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([  3.25742548e-06,   5.76377942e-05,   2.16178814e-05, ...,
         2.38160579e-01,   6.28051171e-05,   8.42752570e-11])

In [33]:
# calculate accuracy
metrics.accuracy_score(y_test, y_pred_class)

0.8098935403826143

In [34]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

ValueError: multiclass format is not supported

In [35]:
# store the vocabulary of X_train
X_train_tokens = vect.get_feature_names()
len(X_train_tokens)

22273

In [36]:
# examine the first 50 tokens
print(X_train_tokens[0:50])

['aafia', 'aah', 'aamaq', 'aamir', 'aan', 'aapl', 'aaron', 'aback', 'abadam', 'abadi', 'abagnale', 'abana', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abarca', 'abassi', 'abated', 'abavley', 'abbess', 'abbie', 'abc', 'abcdedgar', 'abdallah', 'abdan', 'abde', 'abdel', 'abderrahmane', 'abdi', 'abdiqadir', 'abdomen', 'abdominal', 'abdu', 'abduallah', 'abducted', 'abducting', 'abduction', 'abductions', 'abdul', 'abdulkadir', 'abdulla', 'abdullah', 'abdulrahman', 'abetting', 'abhorrent', 'abiad', 'abide', 'abiding', 'abilities']


In [37]:
# examine the last 50 tokens
print(X_train_tokens[-50:])

['zanders', 'zappos', 'zara', 'zarqawi', 'zawahiri', 'zawr', 'zayn', 'zazzle', 'zdyidasqdl', 'zealand', 'zealous', 'zehaf', 'zehafs', 'zeid', 'zeina', 'zeleny', 'zep', 'zeppelin', 'zero', 'zeroed', 'zeroing', 'zeron', 'zeynep', 'zhejiang', 'zhenghua', 'zhou', 'zia', 'ziggy', 'zimmer', 'zimmerman', 'zing', 'zintan', 'zionist', 'zip', 'zipper', 'zipping', 'zm', 'zobl', 'zombies', 'zone', 'zones', 'zoning', 'zoology', 'zor', 'zs', 'zubayr', 'zubeyr', 'zuckerberg', 'zulu', 'zxycaeuwb']


In [38]:
# Naive Bayes counts the number of times each token appears in each class
nb.feature_count_

array([[   0.,    3.,    0., ...,    6.,    1.,   10.],
       [   0.,    1.,    0., ...,    4.,    0.,    0.],
       [  63.,    0.,   17., ...,   20.,    0.,    4.],
       [ 105.,    8.,    0., ...,   27.,   10.,  136.]])

In [39]:
# rows represent classes, columns represent tokens
nb.feature_count_.shape

(4, 22273)

In [40]:
agree_token_count = nb.feature_count_[0, :]
agree_token_count

array([  0.,   3.,   0., ...,   6.,   1.,  10.])

In [41]:
# number of times each token appears across all disagree articles
disagree_token_count = nb.feature_count_[1, :]
disagree_token_count

array([ 0.,  1.,  0., ...,  4.,  0.,  0.])

In [42]:
discuss_token_count = nb.feature_count_[2, :]
discuss_token_count

array([ 63.,   0.,  17., ...,  20.,   0.,   4.])

In [43]:
unrelated_token_count = nb.feature_count_[3, :]
unrelated_token_count

array([ 105.,    8.,    0., ...,   27.,   10.,  136.])

In [44]:
# create a DataFrame of tokens with their separate class counts
tokens = pd.DataFrame({'token':X_train_tokens, 'agree':agree_token_count, 'disagree':disagree_token_count, 'discuss':discuss_token_count, 'unrelated':unrelated_token_count}).set_index('token')
tokens.head()

,agree,disagree,discuss,unrelated
token,,,,
aafia,0.0,0.0,63.0,105.0
aah,3.0,1.0,0.0,8.0
aamaq,0.0,0.0,17.0,0.0
aamir,47.0,0.0,21.0,259.0
aan,0.0,0.0,3.0,5.0


In [45]:
# examine 5 random DataFrame rows
tokens.sample(5, random_state=6)

,agree,disagree,discuss,unrelated
token,,,,
interrogation,10.0,12.0,46.0,189.0
miss,95.0,20.0,137.0,1191.0
faction,11.0,7.0,71.0,134.0
munchies,5.0,3.0,0.0,106.0
moneymaker,0.0,0.0,1.0,0.0


In [47]:
# Naive Bayes counts the number of observations in each class
nb.class_count_

array([  2755.,    618.,   6640.,  27466.])

In [48]:
# add 1 to ham and spam counts to avoid dividing by 0
tokens['agree'] = tokens.agree + 1
tokens['disagree'] = tokens.disagree + 1
tokens['discuss'] = tokens.disagree + 1
tokens['unrelated'] = tokens.disagree + 1
tokens.sample(5, random_state=6)

,agree,disagree,discuss,unrelated
token,,,,
interrogation,11.0,13.0,14.0,14.0
miss,96.0,21.0,22.0,22.0
faction,12.0,8.0,9.0,9.0
munchies,6.0,4.0,5.0,5.0
moneymaker,1.0,1.0,2.0,2.0


In [50]:
# convert the class counts into frequencies
tokens['agree'] = tokens.agree / nb.class_count_[0]
tokens['disagree'] = tokens.disagree / nb.class_count_[1]
tokens['discuss'] = tokens.discuss / nb.class_count_[2]
tokens['unrelated'] = tokens.unrelated / nb.class_count_[3]
tokens.sample(5, random_state=6)

,agree,disagree,discuss,unrelated
token,,,,
interrogation,1.449271e-06,0.000034,3.175352e-07,1.855826e-08
miss,1.264818e-05,0.000055,4.989839e-07,2.916298e-08
faction,1.581022e-06,0.000021,2.041298e-07,1.193031e-08
munchies,7.905112e-07,0.000010,1.134054e-07,6.627949e-09
moneymaker,1.317519e-07,0.000003,4.536217e-08,2.651180e-09


In [51]:
# calculate the ratio of classes for each token
tokens['spam_ratio'] = tokens.unrelated / tokens.agree / tokens.discuss / tokens.disagree
tokens.sample(5, random_state=6)

,agree,disagree,discuss,unrelated,spam_ratio
token,,,,,
interrogation,1.449271e-06,0.000034,3.175352e-07,1.855826e-08,1.184757e+09
miss,1.264818e-05,0.000055,4.989839e-07,2.916298e-08,8.403785e+07
faction,1.581022e-06,0.000021,2.041298e-07,1.193031e-08,1.764795e+09
munchies,7.905112e-07,0.000010,1.134054e-07,6.627949e-09,7.059180e+09
moneymaker,1.317519e-07,0.000003,4.536217e-08,2.651180e-09,1.694203e+11


In [52]:
# examine the DataFrame sorted by spam_ratio
# note: use sort() instead of sort_values() for pandas 0.16.2 and earlier
tokens.sort_values('spam_ratio', ascending=False)

,agree,disagree,discuss,unrelated,spam_ratio
token,,,,,
aafia,1.317519e-07,0.000003,4.536217e-08,2.651180e-09,1.694203e+11
originating,1.317519e-07,0.000003,4.536217e-08,2.651180e-09,1.694203e+11
doomsayers,1.317519e-07,0.000003,4.536217e-08,2.651180e-09,1.694203e+11
doocy,1.317519e-07,0.000003,4.536217e-08,2.651180e-09,1.694203e+11
dont,1.317519e-07,0.000003,4.536217e-08,2.651180e-09,1.694203e+11
leggings,1.317519e-07,0.000003,4.536217e-08,2.651180e-09,1.694203e+11
donjuan,1.317519e-07,0.000003,4.536217e-08,2.651180e-09,1.694203e+11
dongle,1.317519e-07,0.000003,4.536217e-08,2.651180e-09,1.694203e+11
shoved,1.317519e-07,0.000003,4.536217e-08,2.651180e-09,1.694203e+11


In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear', C=1).fit(X_train_dtm, y_train)
clf.score(X_test_dtm, y_test) 

In [ ]:
from sklearn import metrics
scores = cross_val_score(clf, X_test_dtm, y_test, cv=5, scoring='f1_macro')

In [215]:
scores

array([ 0.32343234,  0.434411  ,  0.32552954,  0.32444444,  0.32444444])